# 1. Import Library

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [2]:
sess = tf.InteractiveSession()

# 2. Set Random Seed

In [3]:
random_seed = 2017
np.random.seed(random_seed)
tf.set_random_seed(random_seed)

# 3. Load MNIST Data

In [4]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# test data
test_images = mnist.test.images.reshape(10000, 28, 28, 1)
test_labels = mnist.test.labels[:]

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
print('number of MNIST train data : %5d' % len(mnist.train.images))
print('number of MNIST test data  : %5d' % len(mnist.test.images))

number of MNIST train data : 55000
number of MNIST test data  : 10000


# 4. Data Augmentation
- Rotation (Range = 10, 20, 30)
- Width Shift (Range = 0.1, 0.2, 0.3)

In [6]:
augmentation_size = 1000

In [7]:
images = np.concatenate((mnist.train.images.reshape(55000, 28, 28, 1), mnist.validation.images.reshape(5000, 28, 28, 1)), axis=0)
labels = np.concatenate((mnist.train.labels, mnist.validation.labels), axis=0)

datagen_list = [
                ImageDataGenerator(rotation_range=10),
                ImageDataGenerator(rotation_range=20),
                ImageDataGenerator(rotation_range=30),
                ImageDataGenerator(width_shift_range=0.1),
                ImageDataGenerator(width_shift_range=0.2),
                ImageDataGenerator(width_shift_range=0.3),
               ]

for datagen in datagen_list:
    datagen.fit(images)
    for image, label in datagen.flow(images, labels, batch_size=augmentation_size, shuffle=True, seed=random_seed):
        images = np.concatenate((images, image), axis=0)
        labels = np.concatenate((labels, label), axis=0)
        break

In [8]:
print('final number of train data : %7d' % len(images))
print('images shape : ', images.shape)
print('labels shape : ', labels.shape)

final number of train data :   66000
images shape :  (66000, 28, 28, 1)
labels shape :  (66000, 10)


# 5. Parameter

In [9]:
epochs = 1
conv_filters = [64, 128, 128, 128, 128]
conv_kernerls = [3, 3]
conv_active_fn = tf.nn.elu

dens_units= [1024, 1024, 10]
dens_active_fn = tf.nn.elu
keep_prob = 0.5

learning_rate = 0.00005
batch_size = 256

In [10]:
images = mnist.train.images[:]
labels = mnist.train.labels[:]

# 6. Modeling

### Placeholder

In [11]:
input_data = tf.placeholder(tf.float32, shape=[None, 784])
label_data = tf.placeholder(tf.float32, shape=[None, 10])
train_mode = tf.placeholder(tf.bool)

### Convolutional Layers

In [12]:
conv1_1 = tf.layers.conv2d(inputs=tf.reshape(input_data, [-1, 28, 28, 1]), filters=conv_filters[0], kernel_size=conv_kernerls, 
                         padding='SAME', activation=conv_active_fn, kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
conv1_2 = tf.layers.conv2d(inputs=conv1_1, filters=conv_filters[1], kernel_size=conv_kernerls, 
                         padding='SAME', activation=conv_active_fn, kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
pool1 = tf.layers.max_pooling2d(inputs=conv1_2, pool_size=[2, 2], padding='SAME', strides=2)
dropout1 = tf.layers.dropout(inputs=pool1, rate=keep_prob, training=train_mode)

conv2_1 = tf.layers.conv2d(inputs=dropout1, filters=conv_filters[2], kernel_size=conv_kernerls, 
                         padding='SAME', activation=conv_active_fn, kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
conv2_2 = tf.layers.conv2d(inputs=conv2_1, filters=conv_filters[3], kernel_size=conv_kernerls, 
                         padding='SAME', activation=conv_active_fn, kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
pool2 = tf.layers.max_pooling2d(inputs=conv2_2, pool_size=[2, 2], padding='SAME', strides=2)
dropout2 = tf.layers.dropout(inputs=pool2, rate=keep_prob, training=train_mode)

conv3 = tf.layers.conv2d(inputs=dropout2, filters=conv_filters[4], kernel_size=conv_kernerls, 
                         padding='SAME', activation=conv_active_fn, kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], padding='SAME', strides=2)
dropout3 = tf.layers.dropout(inputs=pool3, rate=keep_prob, training=train_mode)

### Fully Connected Layers

In [13]:
flat = tf.reshape(dropout3, [-1, dropout3.shape[1].value*dropout3.shape[2].value*dropout3.shape[3].value])
dense1 = tf.layers.dense(inputs=flat, units=dens_units[0],activation=dens_active_fn, kernel_initializer=tf.contrib.layers.xavier_initializer())
dropout4 = tf.layers.dropout(inputs=dense1, rate=keep_prob, training=train_mode)
dense2 = tf.layers.dense(inputs=dropout4, units=dens_units[1],activation=dens_active_fn, kernel_initializer=tf.contrib.layers.xavier_initializer())
dropout5 = tf.layers.dropout(inputs=dense2, rate=keep_prob, training=train_mode)

logits = tf.layers.dense(inputs=dropout5, units=dens_units[2],activation=tf.nn.softmax, kernel_initializer=tf.contrib.layers.xavier_initializer())

### Define Cost_fn, Optimizer

In [14]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=label_data))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(label_data, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 7. Training

In [15]:
sess.run(tf.global_variables_initializer())
start_time = datetime.now()
print('Training Start : %s\n' % start_time)
for epoch in range(epochs):
    total_batch = int(len(images) / batch_size)
    for batch in range(total_batch):
        batch_xs, batch_ys = images[batch * batch_size : (batch + 1) * batch_size], labels[batch * batch_size : (batch + 1) * batch_size]
        _, loss, train_acc = sess.run([optimizer, cost, accuracy], feed_dict={input_data: batch_xs, label_data: batch_ys, train_mode: True})
    print(train_acc)


Training Start : 2017-04-30 17:07:26.537082

0.714844


In [16]:
sess.run(accuracy, feed_dict={input_data: mnist.test.images, label_data: mnist.test.labels, train_mode: False})

ResourceExhaustedError: OOM when allocating tensor with shape[10000,64,28,28]
	 [[Node: conv2d_2/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](conv2d/Elu, conv2d_1/kernel/read)]]
	 [[Node: Mean_1/_49 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_57_Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'conv2d_2/convolution', defined at:
  File "C:\Anaconda3\lib\runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Anaconda3\lib\site-packages\traitlets\config\application.